In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import imp
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri

RANDOM_SEED = 0
numpy2ri.activate()
ro.r('set.seed({})'.format(RANDOM_SEED))

import fairtest.utils.log as fairtest_log
imp.reload(fairtest_log)
fairtest_log.set_params(filename='fairtest.log')

In [70]:
import fairtest.investigation as inv
import fairtest.testing as testing
import fairtest.discovery as discovery
import fairtest.error_profiling as error_profiling
import fairtest.modules.metrics as metrics
import fairtest.modules.metrics.correlation as correlation
import fairtest.modules.metrics.regression as regression
import fairtest.modules.metrics.binary_metrics as binary_metrics
import fairtest.modules.statistics.confidence_interval as intervals
import ast
import sklearn.preprocessing as preprocessing
imp.reload(intervals)
imp.reload(inv)
imp.reload(testing)
imp.reload(discovery)
imp.reload(error_profiling)
imp.reload(metrics)
imp.reload(correlation)
imp.reload(binary_metrics)
imp.reload(regression)


dataname = 'adult'
data = pd.read_csv(
            '../data/adult/adult.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('fnlwgt', axis=1)
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Indian' if 'Indian' in x else x, data['race'])
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Black & Other' if ('Indian' in x or 'Other' in x or 'Black' in x) else x, data['race'])
TARGET = 'income'
SENS = ['sex', 'race']
EXPL = []


'''
dataname = 'medical_bin'
data = pd.read_csv(
            '../data/medical/predictions_bin.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''

'''
dataname = 'medical_reg'
data = pd.read_csv(
            '../data/medical/predictions_reg_log.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''

'''
dataname = 'berkeley'
data = pd.read_csv(
            '../data/berkeley/berkeley.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'accepted'
SENS = ['gender']
EXPL = ['department']
#EXPL = []
'''
'''
dataname = 'staples'
data = pd.read_csv(
            '../data/staples/staples.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('zipcode', axis=1)
data = data.drop('distance', axis=1)
data = data.drop('city', axis=1)
TARGET = 'price'
SENS = ['income', 'race']
EXPL = []
'''
'''
data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 
                   'Indian & Alaska Native' if 'Indian' in x  else
                   'Black' if 'Black' in x  else
                   'Pacific Islander' if 'Pacific' in x else
                   'White' if 'White' in x else
                   'Hispanic' if 'Latino' in x else
                   '2+ Races' if 'Two' in x else
                   'Other' if 'Other' in x else
                    x, data['race'])
'''
'''
dataname = 'overfeat'
data = pd.read_csv(
            #'../data/images/lfw_black-white.txt',
            '../data/images/overfeat_raw.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
TARGET = labels.tolist()
'''
'''
dataname = 'overfeat'
data = pd.read_csv(
            '../data/images/overfeat_raw.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
data['Gorilla'] = map(lambda l: 'Yes' if 'chimpanzee' in l else 'No', labeled_data)    
data = data.drop(TARGET, axis=1)
TARGET = 'Gorilla'
'''

'''
dataname = 'benchmark'
data = pd.read_csv(
            '../data/benchmark/benchmark.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'price'
SENS = ['race', 'income']
EXPL = []
'''

#metrics = {'gender': 'Diff'}
metrics = {}

exp = testing.Testing(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
#exp = error_profiling.ErrorProfiling(data, SENS, TARGET, GROUND_TRUTH, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
#exp = discovery.Discovery(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, topk=35, random_state=RANDOM_SEED)
print len(exp.train_set), len(exp.test_set)
exp.train_set.tail()

INFO:root:New Testing Investigation
INFO:root:Encoding Feature Workclass
INFO:root:Encoding Feature education
INFO:root:Encoding Feature marital-status
INFO:root:Encoding Feature occupation
INFO:root:Encoding Feature relationship
INFO:root:Encoding Feature race
INFO:root:Encoding Feature sex
INFO:root:Encoding Feature native-country
INFO:root:Encoding Feature income
INFO:root:Target Feature: Target(names=['income'], arity=2)
INFO:root:Training Size 24421 -  Testing Size 24421
INFO:root:Choosing metric NMI for feature sex
INFO:root:Choosing metric NMI for feature race


24421 24421


,age,Workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
21243,65,0,11,9,2,0,0,4,1,2414,0,30,39,0
45891,61,0,9,13,2,0,0,4,1,4386,0,40,39,1
42613,41,6,15,10,2,10,0,4,1,0,0,60,39,1
43567,47,6,9,13,2,12,0,4,1,0,0,40,39,1
2732,25,4,15,10,4,8,1,4,1,2597,0,48,39,0


In [71]:
import fairtest.modules.context_discovery.guided_tree as guided_tree
imp.reload(guided_tree)
inv.train([exp], score_aggregation='avg', min_leaf_size=100, max_depth=5)

INFO:root:Begin training phase with protected feature sex
INFO:root:Building a Guided Decision Tree
INFO:root:splitting on education-num (score=0.0476168306331) with threshold 11.5 at pred []
INFO:root:splitting on education-num (score=0.043129127137) with threshold 9.5 at pred ['education-num <= 11.5']
INFO:root:splitting on occupation (score=0.0180890050867) with threshold None at pred ['education-num <= 11.5', 'education-num <= 9.5']
INFO:root:splitting on age (score=0.00888757458424) with threshold 46.5 at pred ['education-num <= 11.5', 'education-num <= 9.5', 'occupation = 14']
INFO:root:splitting on relationship (score=0.00645204218585) with threshold None at pred ['education-num <= 11.5', 'education-num <= 9.5', 'occupation = 14', 'age <= 46.5']
INFO:root:splitting on education-num (score=0.00972590227126) with threshold 8.5 at pred ['education-num <= 11.5', 'education-num <= 9.5', 'occupation = 14', 'age > 46.5']
INFO:root:splitting on age (score=0.0297190814467) with threshold

In [6]:
print exp.trained_trees['Age']

KeyError: 'Age'

In [72]:
import fairtest.modules.statistics.hypothesis_test as tests
import fairtest.modules.statistics.multiple_testing as multitest
import fairtest.modules.context_discovery.tree_parser as tree_parser
import fairtest.modules.statistics.confidence_interval as intervals
imp.reload(tests)
imp.reload(multitest)
imp.reload(tree_parser)
imp.reload(intervals)

inv.test([exp], exact=True, prune_insignificant=True)

INFO:root:Begin testing phase
INFO:root:Testing 144 hypotheses


In [73]:
import fairtest.modules.bug_report.report as rep
import fairtest.modules.bug_report.filter_rank as fr
imp.reload(rep)
imp.reload(fr)

output_dir = '../results_new'
inv.report([exp], dataname, output_dir=output_dir, ci_level=0.95, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)